# Import

In [1]:
import sys
import os
import pandas as pd

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
if project_root not in sys.path:
    sys.path.append(project_root)
from src.embedding.chunking import TextChunker
from src.embedding.embedder import Embedder
from src.embedding.vector_store import VectorStoreManager

# Load cleaned complaints

In [3]:
df = pd.read_csv("../data/processed/filtered_complaints.csv")

# Add Complaint ID for metadata traceability

In [4]:
df["ComplaintID"] = df.index

# Chunk the narratives

In [5]:
chunker = TextChunker(chunk_size=256, chunk_overlap=32)
chunks = chunker.chunk_texts(df["Cleaned Narrative"].tolist())

# Prepare metadata for each chunk (ID + product)

In [6]:
metadata = []
for i, row in df.iterrows():
    text_chunks = chunker.splitter.split_text(row["Cleaned Narrative"])
    for chunk in text_chunks:
        metadata.append({
            "complaint_id": row["ComplaintID"],
            "product": row["Product"]
        })

# Embed the chunks

In [7]:
embedder = Embedder()
embeddings = embedder.embed_texts(chunks)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\AI Mastery\Project\Intelligent-Complaint-Analysis-for-Financial-Services\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumsam\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13340 [00:00<?, ?it/s]

# Save to vector store

In [8]:
store = VectorStoreManager(dim=embeddings.shape[1])
store.add_embeddings(embeddings, metadata)
store.save()
print("Vector index and metadata saved.")

Vector index and metadata saved.
